In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sympy
from sympy import isprime, primerange
from tqdm import tqdm
from copy import copy
from matplotlib import collections  as mc
import uuid
from subprocess import call
from multiprocessing import Process
import random



%matplotlib inline

## Utils

In [ ]:
def write_to_file(seq, out):
    with open(out, "w") as f:
        f.write(" ".join([str(x) for x in seq]))

In [ ]:
def read_from_file(inp):
    with open(inp, "r") as f:
        data = f.readlines()[0].strip().split(" ")
        return [int(x) for x in data]

In [ ]:
from math import hypot
class Node:
    """
    represents a node in a TSP tour
    """
    def __init__(self, num, coords):
        self.num = num # start position in a route's order
        self.x = coords[0]   # x coordinate
        self.y = coords[1]   # y coordinate

    def __str__(self):
        """
        returns the string representation of a Node
        """
        return self.num

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def euclidean_dist(self, other):
        """
        returns the Euclidean distance between this Node and other Node
        other - other node
        """
        dx = self.x - other.x
        dy = self.y - other.y
        return hypot(dx, dy)

In [ ]:
def read_from_julia(input_file):
    opts = []
    with open(input_file, "r") as f:
        size = int(f.readline())
        for i in range(size):
            distance,i,j = (x for x in f.readline().strip().split(" "))
            distance = float(distance)
            i = int(i)
            j = int(j)
            opts.append((distance,i,j))
    return opts

In [ ]:
def plot_path(path, coordinates, k=0):
    # Plot tour
    lines = []
    distances = [] 
    for i in range(1, len(path)):
        line = [coordinates[path[i-1]], coordinates[path[i]]]
        (x1, y1), (x2, y2) = line
        distance = np.sqrt((x1-x2)**2 + (y1-y2)**2)
        distances.append(distance)   
        if distance > k:
            lines.append(line)
        
    lc = mc.LineCollection(lines, linewidths=2)
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_aspect('equal')
    plt.grid(False)
    ax.add_collection(lc)
    ax.autoscale()
    return distances

## Prepare input

In [ ]:
cities = pd.read_csv('../input/cities.csv', index_col=['CityId'])

In [ ]:
class Tour(list):  # list with trailing 0
    penalized = ~cities.index.isin(sympy.primerange(0, len(cities)))

    @staticmethod
    def from_file(filename):  # from linkern's output or csv
        seq = [int(x) for x in open(filename).read().split()[1:]]
        return Tour(seq if seq[-1] == 0 else (seq + [0]))

    def score(self):
        df = cities.reindex(self)
        dist = np.hypot(df.X.diff(-1), df.Y.diff(-1))
        penalty = 0.1 * dist[9::10] * self.penalized[self[9::10]]
        _score = dist.sum() + penalty.sum()
        _score = np.round(_score,2)

        return _score

    def to_csv(self, filename):
        pd.DataFrame({'Path': self}).to_csv(filename, index=False)

In [ ]:
tour = Tour.from_file('../../../Downloads/LKH-2.0.9/data/linkern.tour')
#tour.to_csv('pool/{}.csv'.format(tour.score()))
print(tour.score())

## Genetic Operators

### Newly improved GSX (GSX-2).

In [ ]:
def crossover(a,b):
    write_to_file(a, "/tmp/aa.tsp")
    write_to_file(b, "/tmp/bb.tsp")
    
    call(["../cpp/gsx2", "/tmp/aa.tsp", "/tmp/bb.tsp", "/tmp/out.tsp"])
    
    out_seq = read_from_file("/tmp/out.tsp")
    
    assert(len(out_seq)==len(a))
    assert(out_seq[0] == out_seq[-1] == 0)
    
    return out_seq

 ### Double-bridge move mutation operator

In [ ]:
def mutate(indiv):
#    i, ii, j, jj, k, kk, l, ll = 3, 4, 9, 10, 28, 29, 35, 36
    
    i = np.random.randint(1, int(0.25*len(indiv.path)))
    ii = i+1
    
    j = np.random.randint(int(0.25*len(indiv.path))+1, int(0.50*len(indiv.path)))
    jj = j+1
    
    k = np.random.randint(int(0.50*len(indiv.path))+1, int(0.75*len(indiv.path)))
    kk = k+1
    
    l = np.random.randint(int(0.75*len(indiv.path))+1, len(indiv.path)-2)
    ll = l+1
    
    new_genome = indiv.path[0:i+1]

    new_genome.append(indiv.path[kk])
    new_genome.extend(indiv.path[kk+1:l+1])
    new_genome.append(indiv.path[jj])
    new_genome.extend(indiv.path[jj+1:k+1])
    new_genome.append(indiv.path[ii])
    new_genome.extend(indiv.path[ii+1:j+1])
    new_genome.append(indiv.path[ll])
    new_genome.extend(indiv.path[ll+1:])


    assert len(new_genome)==len(indiv.path)
    assert new_genome[0]==new_genome[-1]==0
    assert len(set(new_genome))==len(new_genome)-1
    
    return Individual(new_genome)

## Mock

In [ ]:
def task_julia(start, end, path_to_input, path_to_output, k=20):
    ## Step1: load route 
    route = read_from_file(path_to_input)
    
    ## Evaluate current distance
    distance = Tour(route).score()
    best_distance = distance
    
    ## Perform k-opt between start and end for the k neighbors
    opts = []
    for i in range(start, end):
        for j in range(i+1, len(route)-1):
            new_route = swap_2opt(route, i, j) ## replace by call to Julia
            new_distance = new_route.score()
            if new_distance < best_distance:
                opts.append((new_distance, i, j))

    ## Write solutions to file
    with open(path_to_output, "w") as f:
        f.write(str(len(opts)))
        f.write("\n")
        for opt in opts:
            f.write(str(opt[0]) + " " + str(opt[1]) + " " + str(opt[2]))
            f.write("\n")

## Legacy 

In [ ]:
def swap_2opt(route, i, k):
    """
    swaps the endpoints of two edges by reversing a section of nodes, 
    ideally to eliminate crossovers
    returns the new route created with a the 2-opt swap
    route - route to apply 2-opt
    i - start index of the portion of the route to be reversed
    k - index of last node in portion of route to be reversed
    pre: 0 <= i < (len(route) - 1) and i < k < len(route)
    post: length of the new route must match length of the given route 
    """
    assert i >= 0 and i < (len(route) - 1)
    assert k > i and k < len(route)
    new_route = route[0:i]
    new_route.extend(reversed(route[i:k + 1]))
#    reversed(route[i:k + 1])
    new_route.extend(route[k+1:])
    assert len(new_route)==len(route)
    assert new_route[0]==new_route[-1]==0
    assert len(set(new_route))==len(new_route)-1
    return Tour(new_route)

## Heuristics 

In [ ]:
def run_2opt(route):
    """
    improves an existing route using the 2-opt swap until no improved route is found
    best path found will differ depending of the start node of the list of nodes
        representing the input tour
    returns the best path found
    route - route to improve
    """
    improvement = True
    best_route = route
    best_distance = route.score()
    
    n_threads = 4
    
    cities_per_thread = []
    if (len(route)-2)%n_threads:
        cities_per_thread.append((len(route)-2)%n_threads) 
    else:
        cities_per_thread.append(len(route)//n_threads)
    for i in range(n_threads-1):
        cities_per_thread.append(len(route)//n_threads)

    
    buckets = [] 
    OPTS = []

    for i, bucket in enumerate(np.cumsum(cities_per_thread)):
        #print(i, bucket)
        if i == 0:
            buckets.append((1, bucket))#    population[0].path[1:-1][:bucket]     )

        else:
            buckets.append((buckets[-1][1], bucket)) #population[0].path[1:-1][np.cumsum(cities_per_thread)[i-1]:bucket]  )

    #print(buckets)    
   
    while improvement: 
        improvement = False
        
        processes = []
        file_out = ["/tmp/_raptor_julia_{}.tsp".format(x) for x in range(n_threads)]
        file_in = "/tmp/input_julia.tsp"
        write_to_file(route, file_in)
        best_distance = route.score()
        
        
        for idx, bucket in enumerate(buckets):
            p = Process(target=task_julia, args=(bucket[0], bucket[1], file_in, file_out[idx]))
            processes.append(p)
            p.start()
        for process in processes:
            process.join()

            
        for fic in file_out:
            opts = read_from_julia(fic)
            for opt in opts:
                OPTS.append(opt)
        #print("++++")
        #print(len(OPTS))
        #print("++++")
        
        if len(OPTS) > 0:
            improvement = True
            OPTS = sorted(OPTS, key=lambda x: x[0])
            #print( OPTS[0] )
            if len(OPTS) > 4:
                chosen_swap = random.choice(OPTS[:4])
            else:
                chosen_swap = OPTS[0]
            route = swap_2opt(route, chosen_swap[1], chosen_swap[2])
            best_distance = route.score()
            #print(best_distance)

            OPTS = []
            
       
    assert len(best_route)==len(route)
    assert best_route[0]==best_route[-1]==0
    assert len(set(best_route))==len(best_route)-1
    
    return route

## Hybrid Genetic Algorithm

### Helpers

In [ ]:
def generate_ind():
    indiv = list(range(1,42))
    indiv = np.random.permutation(indiv)
    indiv = np.insert(indiv, 0, 0)
    indiv = np.insert(indiv, len(indiv), 0)
    indiv = list(indiv)
    
    assert indiv[0]==indiv[-1]==0
    assert len(set(indiv))==len(indiv)-1
    
    return Individual(indiv)

In [ ]:
class Individual():
    def __init__(self, path):
        self.path = path
        self.fitness = Tour(path).score()
        self.ranking = 0

In [ ]:
def get_rank(pop_size, selection_pressure, position):
    rank = 2 - selection_pressure + 2*(selection_pressure-1)*((position-1)/(pop_size-1))
    return rank

### Settings

In [ ]:
population = []
popSize = 100
nGen = 50
SP = 2.0
mut_rate = .2
for i in range(popSize):
    ind = generate_ind()
    ind.pos = i
    population.append(ind)

In [ ]:
population = sorted(population, key=lambda x: x.fitness, reverse=True)
for i in range(len(population)):
    population[i].rank = get_rank(popSize, SP, i)

In [ ]:
rnd = uuid.uuid4()
for i in range(nGen):
    population = sorted(population, key=lambda x: x.fitness, reverse=True)
    for j in range(len(population)):
        population[j].rank = get_rank(popSize, SP, j)
    population = sorted(population, key=lambda x: x.rank, reverse=False)
        
    best_score = population[-1].fitness
    write_to_file(seq=population[-1].path, out="./{}_{}_{}.tsp".format(best_score, rnd, i))
    print("Gen {} - Best: {}".format(i, best_score))
    
    
    if np.random.random() < mut_rate:
        parent = population[-1] 
        child = mutate(parent)
    else:
        p1 = population[-1] 
        p2 = population[-2] 
        
        child = crossover(p1.path, p2.path)
        child = Individual(child)

    child.path = run_2opt(Tour(child.path))
    child.fitness = Tour(child.path).score()
    
    print(child.fitness)
    if child.fitness < population[0].fitness:
        population[0] = child
    

In [ ]:
### Best: 20546.52

In [ ]:
distances = plot_path(population[-1].path, cities[["X", "Y"]].values)